<a href="https://colab.research.google.com/github/20247120/IBPSem1Assignment/blob/main/resource_flucuations_simulation_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ISYS2001 Introduction to Business Programming
#### Programming Assignment - Semester 1, 2024
#####*20247120 Cadence Gant*
---

This notebook serves as an interactive report to simulate and analyze the fluctuations in resource prices. It aims to provide a deeper understanding of market dynamics, the impact of supply disruptions, and potential hedging strategies for businesses dealing with critical resources.

## Simulation Overview

This report is intended to demonstrate the effects of altering different variables on a simulation of a normal (or Gaussian) distribution of resource price fluctuations over time (NumPy Developers n.d.). The parameters used in this simulation are as follows, as detailed in [resource_fluctuations_simulation.py](https://github.com/teaching-repositories/simulacra/blob/main/simulacra/resource_fluctuations_simulation.py):

start_price (float)
* the price of the resource at the start of the simulation

days (int)
* the number of days the simulation models

volatility (float)
* standard deviation of the normal distribution of price fluctuations (the 'variance' in percentage changes in price per day, representing volatility in the fluctuation)

drift (float)
* center of the normal distribution of price fluctuations (most likely percentage change in price per day)

supply_disruption_day (int) _Optional_
* an optional day on which supply is disrupted
* must be equal to or less than the number of days the simulation models

disruption_severity (float)
* a multiplicative value increasing prices after a disruption occurs

random_seed (int) _Optional_
* an optional seed for a random number generator to affect the way the normal distribution of price fluctuations is generated



### Baseline Simulation

In this section we want to understand and visualise the fluctuations without the supply disruption day event.

First we need to install necessary packages.




In [ ]:
!pip install git+https://github.com/teaching-repositories/simulacra.git -q

We need to
  - **Import the necessary libraries**: Code cells to import libraries (like `matplotlib` for plotting and any other libraries needed).
  - **Run the Simulation**: Code to run the simulation with zero disruption impact.
  - **Visualise the Results**: Plotting prices over time to establish a baseline.

In [ ]:
from simulacra import ResourceFluctuationsSimulation
import matplotlib.pyplot as plt


# Helper function to display the plot
def plot_prices(prices, supply_disruption_day=None):
    """
    Plots the prices from a simulation with an optional vertical line marking a supply disruption.

    Parameters:
        prices (List[float]): A list of prices to be plotted.
        supply_disruption_day (Optional[int]): The day on which the supply disruption occurs (defaults to None).

    Returns:
        None
    """
    plt.figure(figsize=(10, 6))
    plt.plot(prices, label='Resource Price')
    if supply_disruption_day is not None:
        plt.axvline(x=supply_disruption_day, color='r', linestyle='--', label='Supply Disruption')
    plt.xlabel('Days')
    plt.ylabel('Price')
    plt.title('Resource Price Simulation')
    plt.legend()
    plt.show()

# Setup the simulation
sim = ResourceFluctuationsSimulation(start_price=100, days=250, volatility=0.015,
                             drift=0.0003, supply_disruption_day=100, disruption_severity=0.3)

# Run the simulation
prices = sim.run_simulation()

# Visualise the results
plot_prices(prices, sim.supply_disruption_day)

### Impact of Volatility on Price Stability
Analyze how different volatility settings affect the stability and predictability of resource prices. Discuss the observed trends and implications.


In [ ]:
#import the simulation under a shorter alias for ease of use
from simulacra import ResourceFluctuationsSimulation as rfs
import matplotlib.pyplot as plt
from numpy import random

#copying over the plot display function as it does not need to be edited for these tests
#edited params slightly to include a variable plot title when displaying multiple test results in one output
def plot_prices(plot_title, prices, supply_disruption_day=None):
    """
    Plots the prices from a simulation with an optional vertical line marking a supply disruption.

    Parameters:
        plot_title: A custom title to be displayed on the plot.
        prices (List[float]): A list of prices to be plotted.
        supply_disruption_day (Optional[int]): The day on which the supply disruption occurs (defaults to None).

    Returns:
        None
    """
    plt.figure(figsize=(10, 6))
    plt.plot(prices, label='Resource Price')
    if supply_disruption_day is not None:
        plt.axvline(x=supply_disruption_day, color='r', linestyle='--', label='Supply Disruption')
    plt.xlabel('Days')
    plt.ylabel('Price')
    plt.title(plot_title)
    plt.legend()
    plt.show()

#defining a random seed to be the same in each batch of tests for accurate comparison purposes
rng_seed = random.randint(10000000)

#test 1: increasing volatility by a factor of 10 (0.015 * 10 = 0.15) - expecting drastic price variance
test_title_1 = 'Resource Price Simulation - 10x Price Volatility'
sim_1 = rfs(start_price=100, days=250, volatility=0.15, drift=0.0003, supply_disruption_day=100, disruption_severity=0.3, random_seed=rng_seed)
prices_1 = sim_1.run_simulation()
plot_prices(test_title_1, prices_1, sim_1.supply_disruption_day)
#Result of test 1: Price varies severely on an almost daily basis, causing multiple spikes and valleys in the observed plot.
#Multiple tests with this setting have made it impossible to identify any trends other than the extreme volatility causing
#depression-level inflation to happen with frankly ridiculous frequency.

#test 2: setting volatility to half of its original value (0.015 / 2 = 0.0075) - expecting a more stable trend in price
test_title_2 = 'Resource Price Simulation - 0.5x Price Volatility'
sim_2 = rfs(start_price=100, days=250, volatility=0.0075, drift=0.0003, supply_disruption_day=100, disruption_severity=0.3, random_seed=rng_seed)
prices_2 = sim_2.run_simulation()
plot_prices(test_title_2, prices_2, sim_2.supply_disruption_day)
#Result of test 2: Price starts trending slightly upwards with little variance as expected. Stability continues even after
#supply disruption day causes a steep rise in resource price, as the trend continues within a relatively small margin for the
#remainder of the simulation.

#test 3: price volatility is equal to drift (0.0003) - expecting little if any visible price variance
test_title_3 = 'Resource Price Simulation - Price Volatility = Price Drift'
sim_3 = rfs(start_price=100, days=250, volatility=0.0003, drift=0.0003, supply_disruption_day=100, disruption_severity=0.3, random_seed=rng_seed)
prices_3 = sim_3.run_simulation()
plot_prices(test_title_3, prices_3, sim_3.supply_disruption_day)
#Result of test 3: Unlike the previous tests, running the simulation with this setting multiple times appears to produce nearly
#identical plots - as expected, the standard deviation of the price variance being equal to the mean variance causes almost no
#fluctuation to occur in the simulation, and all runs of this test have produced a gentle, unbroken upwards trend for the entirety
#of the simulation.

### Analyzing the Impact of a Supply Disruption
In this section, model the effects of a supply disruption on resource pricing and market stability. Include code to set up different scenarios with varying supply disruption. Use markers or lines to denote disruption start and assess impact. Overlay plots or perform a Side-by-side comparison to show different impacts of supply disruption.  Discuss the observed trends and implicaitons.

In [ ]:
#imports
from simulacra import ResourceFluctuationsSimulation as rfs
import matplotlib.pyplot as plt
from numpy import random

#Doing the coding a little differently than in the volatility tests, I will be running all tests first and passing the values into 2-dimensional
#list of the price lists, then passing each index into the plotting subroutine to overlay all test trends on a single plot.

#Testing function
def impact_tests(rand_seed, disrupt=None):
  """
  Runs a number of resource price fluctuation simulations with differing supply disruption severity values.

  Parameters:
      disrupt: A specified value for the supply disruption day to occur in all tests (defaults to 100 if empty or outside range of simulation)

  Returns:
      List of price lists generated by each test
  """
  #initialise the return list
  prices_list = []

  #checking if disrupt is in valid range, setting to default if not
  #does not check if disrupt is a number!
  if disrupt is None:
    disrupt = 100
  elif 1 > disrupt > 249:
    disrupt = 100

  #Test 1: Disruption severity doubled
  sim_1 = rfs(start_price=100, days=250, volatility=0.015, drift=0.0003, supply_disruption_day=disrupt, disruption_severity=0.6, random_seed=rand_seed)
  prices_list.append(sim_1.run_simulation())

  #Test 2: Disruption severity halved
  sim_2 = rfs(start_price=100, days=250, volatility=0.015, drift=0.0003, supply_disruption_day=disrupt, disruption_severity=0.15, random_seed=rand_seed)
  prices_list.append(sim_2.run_simulation())

  #Test 3: Disruption severity is equal to price volatility (0.015)
  sim_3 = rfs(start_price=100, days=250, volatility=0.015, drift=0.0003, supply_disruption_day=disrupt, disruption_severity=0.015, random_seed=rand_seed)
  prices_list.append(sim_3.run_simulation())

  return prices_list

#Run the testing function and pass returns to the plotter
disruption_day = 100 #this is unnecessary but here in case it needs to be used for other tests
rng_seed = random.randint(10000000)
test_returns = impact_tests(rng_seed)
control_sim = rfs(start_price=100, days=250, volatility=0.015, drift=0.0003, supply_disruption_day=100, disruption_severity=0.3, random_seed=rng_seed)
control_prices = control_sim.run_simulation()

plt.figure(figsize=(10,6))
plt.axvline(x=disruption_day, color='r', linestyle='--', label='Supply Disruption')
#adding control plot for better visualisation
plt.plot(control_prices, color='g', label='Normal Disruption Severity')
plt.plot(test_returns[0], color='c', label='2x Disruption Severity')
plt.plot(test_returns[1], color='m', label='0.5x Disruption Severity')
plt.plot(test_returns[2], color='y', label='0.05x Disruption Severity')
plt.xlabel('Days')
plt.ylabel('Price')
plt.title('The Effect of Supply Disruption Severity on Resource Price')
plt.legend()
plt.show()

### Exploring Hedging Strategies (Optional)
In this section, evaluate the effectiveness of different hedging strategies to mitigate risks associated with price fluctuations.  Set up various hypothetical scenarios with assumed prices and effects. Use graphs to show strategy effectiveness versus cost.

In [ ]:
#imports
from simulacra import ResourceFluctuationsSimulation as rfs
import matplotlib.pyplot as plt
from numpy import random

#global variable used to assign non-red, non-white colours to plot lines
plot_colours = ['c','m','y','g','b','k']

#combining my strategy from the previous two sections to parameterise this function to overlay multiple datasets
#force return type of None to prevent errors from accidentally assigning this function to a variable
def plot_prices(title, num_tests, prices_list, labels_list, disrupt=None) -> None:
  """
  Plots the prices from multiple simulations with an optional display for a supply disruption line.

  Parameters:
      title: The title to be displayed on the plot
      num_tests: The number of tests in the batch to be plotted
      prices_list: A list containing lists of simulation results
      labels_list: The labels to be assigned to the corresponding list of results
      disrupt: The day of the simulation on which a supply disruption occurs

  Returns:
      None
  """
  plt.figure(figsize=(10,6))
  #iterates through the list of sim results to display a variable amount on the plot
  for i in range(0, num_tests):
    plt.plot(prices_list[i], color=plot_colours[i], label=labels_list[i])
  if disrupt is not None:
    plt.axvline(x=disrupt, color='r', linestyle='--', label='Supply Disruption')
  plt.xlabel('Days')
  plt.ylabel('Price')
  plt.title(title)
  plt.legend()
  plt.show()

def drift_cap() -> None:
  """
  Tests simulations in which price drift is intentionally lowered to mitigate investment risk.

  Parameters:
      None

  Returns:
      None
  """
  #as before, generate same random seed for all tests in this batch
  rng_seed = random.randint(10000000)
  #initialise the lists of results and labels, not to be returned but to be piped directly to the plotting function
  prices_list = []
  labels_list = []

  #Control
  sim_0 = rfs(start_price=100, days=250, volatility=0.015, drift=0.0003, supply_disruption_day=100, disruption_severity=0.3, random_seed=rng_seed)
  prices_list.append(sim_0.run_simulation())
  labels_list.append('1x Drift (Control)')

  #Test 1: Price drift is set to 0, making price less likely to vary daily
  sim_1 = rfs(start_price=100, days=250, volatility=0.015, drift=0, supply_disruption_day=100, disruption_severity=0.3, random_seed=rng_seed)
  prices_list.append(sim_1.run_simulation())
  labels_list.append('0x Drift')

  #Test 2: Price drift is inverted (negative)
  sim_2 = rfs(start_price=100, days=250, volatility=0.015, drift=-0.0003, supply_disruption_day=100, disruption_severity=0.3, random_seed=rng_seed)
  prices_list.append(sim_2.run_simulation())
  labels_list.append('-1x Drift')

  #Plot the tests
  plot_prices('Resource Price Fluctuation with Intentionally Lowered Price Drift', 3, prices_list, labels_list, 100)

def price_and_disrupt() -> None:
  """
  Tests simulations in which the starting resource price and disruption severity are both set to different values.

  Parameters:
      None

  Returns:
      None
  """
  #initialise all function variables
  rng_seed = random.randint(10000000)
  prices_list = []
  labels_list = []

  #Control - No alterations
  sim_0 = rfs(start_price=100, days=250, volatility=0.015, drift=0.0003, supply_disruption_day=100, disruption_severity=0.3, random_seed=rng_seed)
  prices_list.append(sim_0.run_simulation())
  labels_list.append('Control')

  #Test 1: Doubled starting price, halved disruption severity
  sim_1 = rfs(start_price=200, days=250, volatility=0.015, drift=0.0003, supply_disruption_day=100, disruption_severity=0.15, random_seed=rng_seed)
  prices_list.append(sim_1.run_simulation())
  labels_list.append('2x Start Price, 0.5x Disruption Severity')

  #Test 2: Halved starting price, doubled disruption severity
  sim_2 = rfs(start_price=50, days=250, volatility=0.015, drift=0.0003, supply_disruption_day=100, disruption_severity=0.6, random_seed=rng_seed)
  prices_list.append(sim_2.run_simulation())
  labels_list.append('0.5x Start Price, 2x Disruption Severity')

  #Plotting
  plot_prices('Resource Price Fluctation with Inversely Proportional Starting Price & Disruption Severity', 3, prices_list, labels_list, 100)

def chaos() -> None:
  """
  Tests simulations in which the values are ridiculous.

  Parameters:
      None

  Returns:
      None
  """
  #initialise variables
  rng_seed = random.randint(10000000)
  prices_list = []
  labels_list = []

  #Control - To help visualise how bizarre this batch is
  sim_0 = rfs(start_price=100, days=250, volatility=0.015, drift=0.0003, supply_disruption_day=100, disruption_severity=0.3, random_seed=rng_seed)
  prices_list.append(sim_0.run_simulation())
  labels_list.append('Control')

  #Test 1: Volatility is set to 1 (very bad idea!)
  sim_1 = rfs(start_price=100, days=250, volatility=1, drift=0.0003, supply_disruption_day=100, disruption_severity=0.3, random_seed=rng_seed)
  prices_list.append(sim_1.run_simulation())
  labels_list.append('100% Price volatility')

  #Test 2: Supply disruption occurs on day 0 (line not graphed)
  sim_2 = rfs(start_price=100, days=250, volatility=0.015, drift=0.0003, supply_disruption_day=0, disruption_severity=0.3, random_seed=rng_seed)
  prices_list.append(sim_2.run_simulation())
  labels_list.append('Disruption occurs on Day 0')

  #Test 3: Disruption severity set to 1
  sim_3 = rfs(start_price=100, days=250, volatility=0.015, drift=0.0003, supply_disruption_day=100, disruption_severity=1, random_seed=rng_seed)
  prices_list.append(sim_3.run_simulation())
  labels_list.append('100% Disruption severity')

  #Test 4: Too silly to provide any meaningful data
  #sim_4 = rfs(start_price=100, days=250, volatility=1, drift=1, supply_disruption_day=100, disruption_severity=1, random_seed=rng_seed)
  #prices_list.append(sim_4.run_simulation())
  #labels_list.append('100% Everything')

  #Plotting
  plot_prices('Resource Price Fluctutation under Extreme Conditions', 4, prices_list, labels_list, 100)

#Run the tests!
drift_cap()
price_and_disrupt()
chaos()

### Conclusions
#### Price Volatility Testing
Altering the price volatility greatly impacts the expected shape of the price trends, with higher volatility causing a more turbulent price. Interestingly, as price volatility decreases, the impact of supply disruption appears to affect the immediate price more, suggesting an inverse correlation between price volatility and disruption severity.
Additionally, it appears that as the volatility approaches 0, the resource price tends more toward a general upward trend over the course of the simulation - and therefore as it approaches 1, the severity of the fluctuations correlate to a negative price trend.

#### Supply Disruption Impact Testing
As expected, increasing disruption severity drastically increases the change in resource price during and after the supply disruption day, but trends following the disruption day are inconsistent across multiple trends, suggesting that the disruption severity does not have a significant effect on the overall trend of the price fluctuation.
It was observed that, due to the nature of the values being further away from 0, multiplicatively increasing the disruption severity value creates a far greater disparity between the price fluctuation caused than multiplicatively decreasing the value. This is to say that: doubling the disruption severity creates a much larger difference in resource price compared to the control as halving the disruption severity.

#### Hedging Strategy Testing
TBA

### Challenges and Further Research
Due to the nature of the resource fluctuation tests having an element of randomness to them (by virtue of using NumPy's numpy.random.normal() function to determine the distribution of resource prices), multiple tests performed in the same batch that alter only one variable will not display the exact differences without a specified random seed. This was fixed by generating the random seed on a per-batch (rather than per-test) basis.

### References and Additional Resources
1. NumPy Developers. n.d. "numpy.random.normal - NumPy v.1.26 Manual." NumPy
Developers. Accessed May 25th, 2024. [URL.](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html)